In [ ]:
#r "./bin/Debug/net7.0/studies.dll"
#r "./bin/Debug/net7.0/core.fs.dll"
#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

In [ ]:
open FSharp.Data
open Plotly.NET

let trade_csv = "d:\\studies\\03_export_date_ticker_screenerid_gap_outcomes.csv"

let csv = studies.GapStudy.parseTradeOutcomes trade_csv

In [ ]:
let columnChart label keys values =
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
let histogramChart label bins values =
    let chart = 
        Chart.Histogram(
            X = values,
            NBinsX = bins,
            Name = label
        )
    chart.Display() |> ignore

let outputSummary includeDistributionChart (summary:studies.Types.TradeSummary) = 

    // this ensures that the numbers align instead of negative EV taking up more space
    let evPositiveSign =
        match summary.EV with
        | x when x >= 0m -> "+"
        | _ -> ""
        
    printfn "%s%.2f EV -> %.2f%% \t %02.3f%% \t %.2f%% \t %.2f avg gain/loss, %s" 
        evPositiveSign summary.EV (summary.WinPct * 100m) (summary.AvgWin) (summary.AvgLoss) summary.AvgGainLoss summary.StrategyName
        
    match includeDistributionChart with
    | true ->
        histogramChart "Gain Distribution" 40 (summary.Gains)
    | false -> ()
    
let filterNamePairs = [
        ("All", fun (o:studies.Types.TradeOutcomeOutput.Row) -> true)
        ("New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 28)
        ("Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 29)
        ("Gap ups", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp)
        ("Gap ups - new highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 28)
        ("Gap ups - top gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 29)
    ]

let createTradeSummaries outcomes =
    
    let tradesGroupedByStrategy =
        outcomes
        |> Seq.groupBy (fun (t:studies.Types.TradeOutcomeOutput.Row) -> t.Strategy) // all strategies, no filters

    filterNamePairs
    |> Seq.collect (
        fun (filterName,filter) ->

            tradesGroupedByStrategy
            |> Seq.map (
                fun (strategy, trades) ->
                    let filteredTrades = trades |> Seq.filter filter
                    let strategy = filteredTrades |> Seq.head |> fun x -> x.Strategy
                    let name = $"{filterName}, {strategy}"
                    studies.Types.TradeSummary.create name filteredTrades
            )
    )
    

let describeOutcomes includeEVChart includeDistributionChart tradeSummaries =
    
    let groupedBySignal = tradeSummaries |> Seq.groupBy (fun (s:studies.Types.TradeSummary) -> s.StrategyName.Split(",")[0])
    
    groupedBySignal
    |> Seq.iter (
        fun (signal,summaries) ->
            
            printfn $"{signal} [{summaries |> Seq.head |> fun x -> x.Total}]"
            
            let sortedSummaries = summaries |> Seq.sortByDescending (fun summary -> summary.EV)
            
            sortedSummaries |> Seq.iter (fun summary -> outputSummary includeDistributionChart summary)
            
            if includeEVChart then
                let keys = sortedSummaries |> Seq.map (fun summary -> summary.StrategyName)
                let values = sortedSummaries |> Seq.map (fun summary -> summary.EV)
            
                columnChart signal keys values
    )
    
let highlightStrategies numberOfTopRecords tradeSummaries =
    
    let sortFunctions = [
        ("EV", fun (t:studies.Types.TradeSummary) -> t.EV)
        ("Win %", fun (t:studies.Types.TradeSummary) -> t.WinPct)
        ("Avg Gain", fun (t:studies.Types.TradeSummary) -> t.AvgGainLoss)
    ]

    sortFunctions
    |> Seq.iter (fun (sortLabel, sortFunction) ->
        printfn $"Top {numberOfTopRecords} by: {sortLabel}"
        tradeSummaries
        |> Seq.sortByDescending sortFunction
        |> Seq.take numberOfTopRecords
        |> Seq.iter (fun summary -> summary |> outputSummary false)
        printfn ""
        printfn ""
    )

In [ ]:
let studiesDirectory = "d:\\studies"
let signalPath = "d:\\studies\\02_export_date_ticker_screenerid_gap.csv"
let priceFunc = studies.DataHelpers.getPricesFromCsv studiesDirectory

let runStrategies strategies = async {
    let! signalsWithPrices = studies.Trading.prepareSignalsForTradeSimulations signalPath priceFunc
    
    return! studies.Trading.runTrades signalsWithPrices strategies
}

let strategies = 
        [
            studies.TradingStrategies.buyAndHoldStrategy None
            studies.TradingStrategies.buyAndHoldStrategy (Some 5)
            studies.TradingStrategies.buyAndHoldStrategy (Some 15)
            studies.TradingStrategies.buyAndHoldStrategy (Some 30)
            studies.TradingStrategies.buyAndHoldStrategy (Some 60)
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false None 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 5) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 15) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 30) 5m
            studies.TradingStrategies.buyAndHoldStrategyWithStopLoss false (Some 60) 5m
        ]

let outcomes = 
    strategies
    |> runStrategies
    |> Async.RunSynchronously
    |> Seq.toList

printfn "Finished running trades, generated:"
printfn $"{outcomes |> Seq.length} trade outcomes"

printfn "Generating trade summaries..."
let tradeSummaries = outcomes |> createTradeSummaries |> Seq.toList
printfn $"{tradeSummaries.Length} summaries generated"

In [ ]:
// now, let's group by signal and see what we got
tradeSummaries
|> describeOutcomes true false

In [ ]:
tradeSummaries
|> List.filter (fun t -> t.EV > 0)
|> describeOutcomes false false

In [ ]:
tradeSummaries
|> highlightStrategies 10

In [ ]:
// sanity check, print random trades, look over them, investigate specific trades if needed

let outcomeDescription (x:studies.Types.TradeOutcomeOutput.Row) =
    printfn $"{x.Ticker} {x.Strategy}: {x.Opened.ToShortDateString()} @ {x.OpenPrice}, {x.Closed.ToShortDateString()} @ {x.ClosePrice}: profit of {x.PercentGain}"
    
let random = System.Random()

outcomes
|> Seq.sortBy (fun x -> random.NextInt64())
|> Seq.take 10
|> Seq.iter outcomeDescription

printfn ""
printfn "Max gain trade with stop loss"

outcomes
|> Seq.filter (fun x -> x.Strategy.Contains("with Stop Loss"))
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

printfn ""
printfn "Max gain overall"

outcomes
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

// investigating what happened with the trade and why it went as it went
let signalsWithPrices = studies.Trading.prepareSignalsForTradeSimulations signalPath priceFunc |> Async.RunSynchronously

let tickerOfInterest = "MLTX"

let signal, prices = 
    signalsWithPrices
    |> Seq.filter (fun (signal, prices) -> signal.Ticker = tickerOfInterest)
    |> Seq.head
    
let strategy = studies.TradingStrategies.buyAndHoldStrategyWithStopLoss true (Some 5) 5m

(signal, prices) |> strategy |> outcomeDescription

In [ ]:
Plotly.NET.Defaults.DefaultWidth <- 900

let metrics = [
    ("EV", fun (s:studies.Types.TradeSummary) -> s.EV)
    ("Win/Loss", fun s -> s.AvgGainLoss)
    ("Win %", fun s -> s.WinPct)
    ("Avg Win", fun s -> s.AvgWin)
    ("Avg Loss", fun s -> s.AvgLoss)
    ("Number of Trades", fun s -> s.Total)
]

metrics
|> List.iter( fun (label,metricFunc) ->

    printfn $"sorting for {label}..."
    
    let sorted = tradeSummaries |> Seq.sortByDescending metricFunc |> Seq.toList
    
    let values = sorted |> List.map metricFunc
    let keys = sorted |> List.map (fun summary -> summary.StrategyName)

    printfn $"mapped keys and values for {label}, charting..."
    
    values |> List.zip keys |> List.iter (fun (l,v) -> printfn $"{l}: {v}")
    
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
    printfn ""
)

In [ ]:
tradeSummaries
|> List.filter ( fun s -> s.StrategyName.Contains("Gap ups - new highs, Buy and Hold"))
|> List.sortByDescending ( fun s -> s.EV)
|> List.iter (fun s -> s |> outputSummary false)

In [ ]:
// let's take only the trades where EV is positive
tradeSummaries
|> List.filter (fun s -> s.EV > 0m)
|> List.sortByDescending (fun s -> s.EV)
|> List.iter (fun s -> outputSummary false s)